GOAL:

- Create points using JSON URL from Open Data site
- Publish the points to ArcGIS Online or ArcGIS Portal

Inspired by:

- [Bryan McIntosh - Export ArcGIS Server Map Service Layer to Shapefile](http://www.spatialtimes.com/2016/03/extract-map-service-layer-shapefile-using-python/)
- [Corey Schafer python JSON YouTube video](https://youtu.be/9N6a-VLBa2I)
- [Corey Schafer GitHub](https://github.com/CoreyMSchafer/code_snippets/blob/master/Python-JSON/api.py)

In [1]:
import urllib3
import json
import arcpy
import pandas as pd
import numpy as np

[urllib3 SSL Warnings](https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings)

In [2]:
urllib3.disable_warnings()

In [3]:
#for landlord permits
jsonURL = "https://data.cityofgainesville.org/resource/s829-win5.json"
gdb = r"C:\Users\friendde\Documents\ArcGIS\Projects\MIMS\MIMS.gdb"
#fc = r"C:\Users\friendde\Documents\ArcGIS\Projects\MIMS\MIMS.gdb\ActiveBusiness\ActiveBusinessCity"
tbl = r"C:\Users\friendde\Documents\ArcGIS\Projects\MIMS\MIMS.gdb\LandlordCity"
#flds = ['BusinessName','BusinessPhone','BusinessType','BusinessAddress','BusinessOwner','POINT_X','POINT_Y','SHAPE@XY']
#flds = ['BusinessName','BusinessPhone','BusinessType','BusinessAddress','BusinessOwner','POINT_X','POINT_Y']
flds = ['Address','ContactName','LandlordLicenseYear','Location_Address','Location_City','Location_State','Location_ZipCode',
         'OwnerName','ParcelNumber','ParcelUnit','ZipCode','POINT_X','POINT_Y']
location = [0,0]
tupleXY = (0,0)

In [4]:
# interagate data
#df = pd.read_json(jsonURL)
#df.tail()
#list(df)

In [5]:
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True
#arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)

In [6]:
def getCoords(_dict):
    if _dict is None:
        return [0,0]
    else:
        #print(_dict.get('coordinates'))
        return _dict.get('coordinates')

In [7]:
http = urllib3.PoolManager()
response = http.request('GET', jsonURL)
#response.status
#response.data

In [8]:
data = json.loads(response.data)

In [9]:
print(json.dumps(data, indent=2))

[
  {
    "parcelunit": "06398-032-000-1",
    "parcelnumber": "06398-032-000",
    "landlordlicenseyear": "2019",
    "parceladdress": "03825 NW 14TH  PL",
    "latitude": "29.665536",
    "longitude": "-82.381663",
    "location": {
      "type": "Point",
      "coordinates": [
        -82.381663,
        29.665536
      ]
    },
    "parcel_city": "GAINESVILLE",
    "location_state": "FL",
    "location_zip": "32605",
    ":@computed_region_ecgy_hwrz": "5",
    ":@computed_region_e6r8_dw75": "35",
    ":@computed_region_axii_i744": "3",
    ":@computed_region_u9vc_vmbc": "6",
    ":@computed_region_4rat_gsiv": "770"
  },
  {
    "parcelunit": "05979-054-000-1",
    "parcelnumber": "05979-054-000",
    "landlordlicenseyear": "2015",
    "parceladdress": "8630 NW 38TH  CIR",
    "parcel_city": "GAINESVILLE",
    "location_state": "FL",
    "location_zip": "32653"
  },
  {
    "parcelunit": "05977-301-000-1",
    "parcelnumber": "05977-301-000",
    "landlordlicenseyear": "2017",
    "

In [10]:
arcpy.TruncateTable_management(tbl)

<Result 'C:\\Users\\friendde\\Documents\\ArcGIS\\Projects\\MIMS\\MIMS.gdb\\LandlordCity'>

['Address','ContactName','LandlordLicenseYear','Location','Location_Address','Location_City','Location_State','Location_Zip',
         'OwnerName','ParcelNumber','ParcelUnit','ZipCode','POINT_X','POINT_Y']
'address',
 'contactname',
 'landlordlicenseyear',
 'location',
 'location_address',
 'location_city',
 'location_state',
 'location_zip',
 'ownername',
 'parcelnumber',
 'parcelunit',
 'zipcode'

In [11]:
# use try/except to catch keyerror
for item in data:
    try:
        cursor = arcpy.da.InsertCursor(tbl,flds)
        _address = item['address']
        _contactname = item['contactname']
        _landlordlicenseyear = item['landlordlicenseyear']
        _location = getCoords(item['location'])
        _location_address = item['location_address']
        _location_city = item['location_city']
        _location_state = item['location_state']
        _location_zip = item['location_zip']
        _ownername = item['ownername']
        _parcelnumber = item['parcelnumber']
        _parcelunit = item['parcelunit']
        _zipcode = item['zipcode']        
        locationX = _location[0]
        locationY = _location[1]
        tupleXY = (locationX,locationY)
        #print(bizname,'\n','\t',bizphone,'\n','\t',biztype,'\n','\t',bizaddress,'\n','\t',locationX,locationY)
        #business = [bizname,bizphone,biztype,bizaddress,bizowner,locationX,locationY,tupleXY]
        landlord = [_address,_contactname,_landlordlicenseyear,_location_address,_location_city,_location_state,_location_zip,
                    _ownername,_parcelnumber,_parcelunit,_zipcode,locationX,locationY]
        print(landlord)
        cursor.insertRow(landlord)
        del cursor
    except KeyError:
        location = [0,0]
        tupleXY = (0,0)
        continue


### Use numpy and pandas to export to CSV
 
Use arcpy [```TableToNumPyArray()```](http://pro.arcgis.com/en/pro-app/arcpy/data-access/tabletonumpyarray.htm)
See also [Working with numpy in ArcGIS](http://pro.arcgis.com/en/pro-app/arcpy/get-started/working-with-numpy-in-arcgis.htm)


#nparr = arcpy.da.TableToNumPyArray(tbl,flds,skip_nulls=True)
#pdarr = pd.DataFrame(nparr)
#pdarr.to_csv(custAcctFile,header=False, index=False)

Ready new numpy array for consumption analysis

In [12]:
nparr = arcpy.da.TableToNumPyArray(tbl,flds,skip_nulls=True)
df = pd.DataFrame(nparr)
df.head(10)

,Address,ContactName,LandlordLicenseYear,Location_Address,Location_City,Location_State,Location_ZipCode,OwnerName,ParcelNumber,ParcelUnit,ZipCode,POINT_X,POINT_Y


In [13]:
# find owners with more than one permitted location
df.groupby('OwnerName')['OwnerName'].value_counts().loc[lambda x : x>1].sort_values(ascending=False)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 1

#pivot table
pivotTbl = df.pivot_table(values=["BusinessType"], index=["BusinessType"], aggfunc='count')
pivotTbl

[```gis.features.SpatialDataFrame()```](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#arcgis.features.SpatialDataFrame.from_xy)

In [ ]:
from arcgis.features import SpatialDataFrame
from arcgis.gis import GIS
from getpass import getpass
from IPython.display import display

In [ ]:
sdf = SpatialDataFrame.from_xy(df,"POINT_X","POINT_Y")
display(sdf)

In [ ]:
#gis = GIS("https://wms.gru.com/portal", username=input("Enter User Name "), password=(getpass()))
gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
#gis = GIS()
#portalDesc = arcpy.GetPortalDescription()
# search and list all items owned by connected user
#query=f'owner:{portalDesc["user"]["username"]} AND title:CW BaseMap'
#itemType="Feature Layer"
#sortField="title"
#sortOrder="asc"
# default max__items is 10
#maxItems=100
#m = gis.content.search(query,itemType,sortField,sortOrder,maxItems)


In [ ]:
landlordLayer = gis.content.import_data(sdf)

In [ ]:
m = gis.map('Gainesville,FL')

In [ ]:
m

In [ ]:
m.add_layer(landlordLayer)
m

In [ ]:
m.display